In [1]:
import os
import pandas as pd, numpy as np
from PIL import Image
from keras.models import Model,Sequential
from keras.layers import Dense,Conv2D,BatchNormalization,Flatten,MaxPool2D,Activation,Dropout
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam,SGD
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
!ls ../input/code-marathon-ml/data/data/

test  test.csv	train  train.csv


In [3]:
df= pd.read_csv('../input/code-marathon-ml/data/data/train.csv')
print(df.head(5))
print(df.count())

  image_name  n_sides
0      0.png        5
1      1.png        8
2      2.png        4
3      3.png        7
4      4.png        4
image_name    18432
n_sides       18432
dtype: int64


In [4]:
df = df.sample(frac=1).reset_index(drop=True)
print(df.head())

  image_name  n_sides
0   5290.png        3
1  14022.png        3
2   5747.png        5
3  14163.png        5
4   4841.png        4


In [5]:
!ls ../input/code-marathon-ml/data/data/

test  test.csv	train  train.csv


In [6]:
model = Sequential()
model.add(Conv2D(32,(5,5),input_shape=(256,256,3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPool2D((3,3)))
model.add(Conv2D(64,(5,5)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPool2D((3,3)))


model.add(Flatten())
model.add(Dropout(0.2))
#model.add(Dense(1))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 252, 252, 32)      2432      
_________________________________________________________________
batch_normalization_1 (Batch (None, 252, 252, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 252, 252, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 84, 84, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 80, 80, 64)        51264     
_________________________________________________________________
batch_normalization_2 (Batch (None, 80, 80, 64)        256       
_________________________________________________________________
activation_2 (Activation)    (None, 80, 80, 64)        0         
__________

,
total size: 18432
train: 16432
test: 2000

In [7]:
path = '../input/code-marathon-ml/data/data/train/'
file_names= df['image_name']
def train_generator(batch_size):
    count=0
    while True:
        X=[]
        y=[]
        line_count=0
        for file_name in file_names[:16432]:
            line_count+=1
            img_path = path+file_name
            img= image.load_img(img_path)
            img= image.img_to_array(img)
            img= img/255
            X.append(img)
            y.append(df.loc[df['image_name']== file_name].iloc[0,1])
            count+=1
            if count==batch_size or line_count == 16432 :
                X= np.array(X)
                y= np.array(y)
                yield [X,y]
                X=[]
                y=[]
                count=0
               


In [8]:
path = '../input/code-marathon-ml/data/data/train/'
file_names= df['image_name']
def val_generator(batch_size):
    count=0
    while True:
        X=[]
        y=[]
        line_count=0
        for file_name in file_names[16432:]:
            line_count+=1
            img_path = path+file_name
            img= image.load_img(img_path)
            img= image.img_to_array(img)
            img= img/255
            X.append(img)
            y.append(df.loc[df['image_name']== file_name].iloc[0,1])
            count+=1
            if count==batch_size or line_count == 2000 :
                X= np.array(X)
                y= np.array(y)
                yield [X,y]
                X=[]
                y=[]
                count=0
               


In [9]:
generator= train_generator(10)
X,y= next(generator)
print(y)
print(X.shape)
print(y.shape)

[3 3 5 5 4 8 3 4 7 3]
(10, 256, 256, 3)
(10,)


In [10]:
X,y= next(generator)
print(y)
print(X.shape)
print(y.shape)


[5 5 8 5 5 3 3 7 6 3]
(10, 256, 256, 3)
(10,)


In [11]:
filepath="./model-{epoch:02d}-{val_loss:.4f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss',save_best_only=True, verbose=1, mode='min')
lr = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.4, min_lr=0.00001)

In [12]:
epochs = 200
batch_size= 128
train_gen= train_generator(batch_size)
val_gen= val_generator(batch_size)
train_rows= 16432
val_rows= 2000
train_steps= int(np.ceil(train_rows/batch_size))
val_steps= int(np.ceil(val_rows/batch_size))
# print(train_steps)
# print(val_steps)
history = model.fit_generator(generator=train_gen,steps_per_epoch= train_steps,validation_data=val_gen,callbacks=[checkpoint,lr], validation_steps=val_steps, epochs=epochs,verbose=1)
model.save('./model.h5')
output_data= pd.DataFrame.from_dict(history.history)
output_data.to_csv('./loss_measuress.csv')

Epoch 1/200
129/129 [==============================] - 139s 1s/step - loss: 24.4846 - val_loss: 2.3189

Epoch 00001: val_loss improved from inf to 2.31885, saving model to ./model-01-2.3189.h5
Epoch 2/200
129/129 [==============================] - 115s 895ms/step - loss: 2.3388 - val_loss: 1.9604

Epoch 00002: val_loss improved from 2.31885 to 1.96043, saving model to ./model-02-1.9604.h5
Epoch 3/200
129/129 [==============================] - 116s 902ms/step - loss: 2.2520 - val_loss: 1.9049

Epoch 00003: val_loss improved from 1.96043 to 1.90494, saving model to ./model-03-1.9049.h5
Epoch 4/200
129/129 [==============================] - 116s 902ms/step - loss: 2.1835 - val_loss: 1.8833

Epoch 00004: val_loss improved from 1.90494 to 1.88333, saving model to ./model-04-1.8833.h5
Epoch 5/200
129/129 [==============================] - 117s 903ms/step - loss: 2.1427 - val_loss: 1.8675

Epoch 00005: val_loss improved from 1.88333 to 1.86751, saving model to ./model-05-1.8675.h5
Epoch 6/200